In [1]:
import pandas as pd

df_asset = pd.read_parquet(f'../sectors/QQQ_daily.parquet')
df_asset

,open,high,low,close,adj close,volume,symbol
Date,,,,,,,
2009-11-16,44.200001,44.650002,44.119999,44.459999,38.832016,87776100,QQQ
2009-11-17,44.340000,44.599998,44.250000,44.599998,38.954292,81884000,QQQ
2009-11-18,44.459999,44.490002,44.049999,44.349998,38.735943,100240800,QQQ
2009-11-19,44.049999,44.080002,43.349998,43.660000,38.133293,114558000,QQQ
2009-11-20,43.430000,43.549999,43.279999,43.439999,37.941128,70912100,QQQ
...,...,...,...,...,...,...,...
2024-11-05,487.609985,492.880005,487.519989,492.209991,492.209991,24353600,QQQ
2024-11-06,500.559998,506.410004,499.600006,505.579987,505.579987,43082200,QQQ
2024-11-07,508.399994,514.330017,508.339996,513.539978,513.539978,32853100,QQQ


In [18]:
def calc_returns(srs: pd.Series, day_offset: int = 1) -> pd.Series:
    """for each element of a pandas time-series srs,
    calculates the returns over the past number of days
    specified by offset

    Args:
        srs (pd.Series): time-series of prices
        day_offset (int, optional): number of days to calculate returns over. Defaults to 1.

    Returns:
        pd.Series: series of returns
    """
    returns = srs / srs.shift(day_offset) - 1.0
    return returns

df_asset = pd.read_parquet(f'../sectors/QQQ_daily.parquet')
df_asset["daily_returns"] = calc_returns(df_asset["close"])

# take a long position, just stay in the market all the time
df_asset["long_position"] = df_asset["daily_returns"].shift(-1).apply(lambda daily_return: 1)
df_asset['long_return'] = df_asset["daily_returns"].shift(-1) * df_asset["long_position"]

# pretend to know the daily return in advance and implement the optimal long/short trading strategy
df_asset["perfect_trader_position"] = df_asset["daily_returns"].shift(-1).apply(lambda daily_return: 1 if daily_return > 0 else -1)
df_asset['perfect_trader_return'] = df_asset["daily_returns"].shift(-1) * df_asset["perfect_trader_position"]

long_total_return = df_asset['long_return'].sum()
print(f"Total return: {long_total_return}")

perfect_trader_total_return = df_asset['perfect_trader_return'].sum()
print(f"Total return: {perfect_trader_total_return}")

df_asset

Total return: 2.7599860925602977
Total return: 33.844167831519854


,open,high,low,close,adj close,volume,symbol,daily_returns,long_position,long_return,perfect_trader_position,perfect_trader_return
Date,,,,,,,,,,,,
2009-11-16,44.200001,44.650002,44.119999,44.459999,38.832016,87776100,QQQ,NaN,1,0.003149,1,0.003149
2009-11-17,44.340000,44.599998,44.250000,44.599998,38.954292,81884000,QQQ,0.003149,1,-0.005605,-1,0.005605
2009-11-18,44.459999,44.490002,44.049999,44.349998,38.735943,100240800,QQQ,-0.005605,1,-0.015558,-1,0.015558
2009-11-19,44.049999,44.080002,43.349998,43.660000,38.133293,114558000,QQQ,-0.015558,1,-0.005039,-1,0.005039
2009-11-20,43.430000,43.549999,43.279999,43.439999,37.941128,70912100,QQQ,-0.005039,1,0.016114,1,0.016114
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-05,487.609985,492.880005,487.519989,492.209991,492.209991,24353600,QQQ,0.012757,1,0.027163,1,0.027163
2024-11-06,500.559998,506.410004,499.600006,505.579987,505.579987,43082200,QQQ,0.027163,1,0.015744,1,0.015744
2024-11-07,508.399994,514.330017,508.339996,513.539978,513.539978,32853100,QQQ,0.015744,1,0.001168,1,0.001168
